In [1]:
import pandas as pd
import numpy as np
import json
import gmaps
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import requests
from census import Census
from config import gkey
from IPython.display import display
import ipywidgets as widgets
import gmaps.geojson_geometries

gmaps.configure(api_key=gkey)


In [2]:
file = "../Resources/Austin Data Set CSV.csv"
base_df = pd.read_csv(file)

In [3]:
mask = base_df.lotSizeSqFt != 1508482800.0
new = base_df[mask]

mask = new.numOfBathrooms != 27.0
new1 = new[mask]

mask = new1.homeType == "Single Family"
cleaned_data = new1[mask]
cleaned_data.homeType.value_counts()

Single Family    14239
Name: homeType, dtype: int64

In [4]:
sale_year = cleaned_data[["latest_saleyear", "latitude", 'longitude', "latestPrice"]]
sale_year.rename(columns= {'latest_saleyear': 'year'}, inplace=True)
sale_month = cleaned_data[["latest_salemonth", "latitude", 'longitude', "latestPrice"]]
sale_month.rename(columns= {'latest_salemonth': 'Month'}, inplace=True)
sale_month

sale_month


C:\Users\arnol\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Month,latitude,longitude,latestPrice
0,9,30.430632,-97.663078,305000
1,10,30.432672,-97.661697,295000
2,7,30.409748,-97.639771,256125
3,8,30.432112,-97.661659,240000
4,10,30.437368,-97.656860,239900
...,...,...,...,...
15165,11,30.185915,-97.828415,280000
15166,12,30.161997,-97.816345,330000
15167,12,30.240223,-97.772835,550000
15168,1,30.244835,-97.761124,875000


In [5]:

class Interactive(object): 
    
    def __init__(self, sale_year):
        self._df = sale_year
        self._heatmap = None
        self._slider = None
        initial_year = min(self._df['year'])

        title_widget = widgets.HTML(
            '<h3>Home sales by year</h3>''<h4>Data from The Austin Housing Project</a></h4>'
            )


        map_figure = self._render_map(initial_year)
        controls = self._render_controls(initial_year)
        self._container = widgets.VBox([title_widget, controls, map_figure])

    def render(self):
        display(self._container)

    def _on_year_change(self, change):
        year = self._slider.value
        self._heatmap.locations = self._locations_for_year(year)
        self._total_box.value = self._total_sales_text_for_year(year)
        return self._container

    def _render_map(self, initial_year):
        fig = gmaps.figure(map_type='HYBRID')
        self._heatmap = gmaps.heatmap_layer(
            self._locations_for_year(initial_year),
            max_intensity=25,
            point_radius=20
        )
        fig.add_layer(self._heatmap)
        return fig

    def _render_controls(self, initial_year):
        self._slider = widgets.IntSlider(
            value=initial_year,
            min=min(self._df['year']),
            max=max(self._df['year']),
            description='Year',
            continuous_update=False
        )
        self._total_box = widgets.Label(
            value=self._total_sales_text_for_year(initial_year)
        )
        self._slider.observe(self._on_year_change, names='value')
        controls = widgets.HBox(
            [self._slider, self._total_box],
            layout={'justify_content': 'space-between'}
        )
        return controls

    def _locations_for_year(self, year):
        return self._df[self._df['year'] == year][['latitude', 'longitude']]

    def _total_sales_for_year(self, year):
        return int(self._df[self._df['year'] == year]['year'].count())

    def _total_sales_text_for_year(self, year):
        return '{} Total Home Sales'.format(self._total_sales_for_year(year))


Interactive(sale_year).render()

In [6]:
class Interactive(object): 
    
    def __init__(self, sale_month):
        self._df = sale_month
        self._heatmap = None
        self._slider = None
        initial_month = min(self._df['Month'])

        title_widget = widgets.HTML(
            '<h3>Home sales by Month</h3>''<h4>Data from The Austin Housing Project</a></h4>'
            )


        map_figure = self._render_map(initial_month)
        controls = self._render_controls(initial_month)
        self._container = widgets.VBox([title_widget, controls, map_figure])

    def render(self):
        display(self._container)

    def _on_month_change(self, change):
        Month = self._slider.value
        self._heatmap.locations = self._locations_for_month(Month)
        self._total_box.value = self._total_sales_text_for_month(Month)
        return self._container

    def _render_map(self, initial_month):
        fig = gmaps.figure(map_type='HYBRID')
        self._heatmap = gmaps.heatmap_layer(
            self._locations_for_month(initial_month),
            max_intensity=25,
            point_radius=25
        )
        fig.add_layer(self._heatmap)
        return fig

    def _render_controls(self, initial_month):
        self._slider = widgets.IntSlider(
            value=initial_month,
            min=min(self._df['Month']),
            max=max(self._df['Month']),
            description='Month',
            continuous_update=False
        )
        self._total_box = widgets.Label(
            value=self._total_sales_text_for_month(initial_month)
        )
        self._slider.observe(self._on_month_change, names='value')
        controls = widgets.HBox(
            [self._slider, self._total_box],
            layout={'justify_content': 'space-between'}
        )
        return controls

    def _locations_for_month(self, Month):
        return self._df[self._df['Month'] == Month][['latitude', 'longitude']]

    def _total_sales_for_month(self, Month):
        return int(self._df[self._df['Month'] == Month]['Month'].count())

    def _total_sales_text_for_month(self, Month):
        return '{} historical Home Sales by Month'.format(self._total_sales_for_month(Month))


Interactive(sale_month).render()